In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.path.dirname("__file__"), "..", "utilities")))
from format import process_federated_dataset,process_specific_datasets_and_files

# federated_endpoint_sets_to_test = ["Uniprot"]
# # Process the federated dataset and get the output directory
# output_dir = process_federated_dataset(
#     endpoint_sets_to_test=federated_endpoint_sets_to_test
# )

# print(f"Processing complete. Results stored in: {output_dir}")

In [9]:
result_dir = process_specific_datasets_and_files(
    endpoint_files_map={"Uniprot": ["48_glycosylation_sites_and_glycans.ttl", "50_tissues_where_genes_sphingosines_are_expressed.ttl"]}
)

print(f"Processing complete. Results stored in: {result_dir}")

Processing endpoint set: Uniprot

Processed: 50_tissues_where_genes_sphingosines_are_expressed.ttl

Processed: 48_glycosylation_sites_and_glycans.ttl
Processed 2 files from Uniprot
Saved metadata for 2 files from 1 endpoint sets to:
  - /Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent/experiments/eval/ev_2025-04-07_13-57-51/testset_meta_data.json
Processing complete. Results stored in: /Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent/experiments/eval/ev_2025-04-07_13-57-51


In [6]:
from datasets import Dataset
from format import load_data_from_file

test_set = load_data_from_file(os.path.join(output_dir, "testset_meta_data.json"))

test_dataset = Dataset.from_dict(test_set)

print(test_dataset[0:1])

{'resource': ['https://sparql.uniprot.org/.well-known/sparql-examples/118_biosodafrontend_rat_brain_human_cancer'], 'natural_language_question': ['What are the Homo sapiens genes associated with cancer and their orthologs expressed in the Rattus norvegicus brain?'], 'query': ['PREFIX up:<http://purl.uniprot.org/core/>\nPREFIX taxon:<http://purl.uniprot.org/taxonomy/>\nPREFIX rdfs:<http://www.w3.org/2000/01/rdf-schema#>\nPREFIX orth:<http://purl.org/net/orth#>\nPREFIX dcterms:<http://purl.org/dc/terms/>\nPREFIX obo:<http://purl.obolibrary.org/obo/>\nPREFIX lscr:<http://purl.org/lscr#>\nPREFIX genex:<http://purl.org/genex#>\nPREFIX sio: <http://semanticscience.org/resource/>\nSELECT ?gene ?orthologous_protein2 WHERE {\n  {\n    SELECT ?protein1 WHERE {\n      ?protein1 a up:Protein;\n        up:organism/up:scientificName \'Homo sapiens\' ;\n        up:annotation ?annotation .\n      ?annotation rdfs:comment ?annotation_text.\n      ?annotation a up:Disease_Annotation .\n      FILTER CONT

In [ ]:
from langsmith import Client
from datetime import datetime

client = Client()
runs = client.list_runs(project_name="sparql-rag-agent", is_root=True)
first_run = next(runs)


child_runs = list(client.list_runs(project_name="sparql-rag-agent", parent_run_id=first_run.id))

#print(child_runs)

# Filter to find the sparql_query_construction run
sparql_construction_run = None
for run in child_runs:
    if run.name == "sparql_query_construction":
        sparql_construction_run = run
        break

if sparql_construction_run:
    result = {
        # Your existing fields...
        "final_state_response": "",
        "run_id_langsmith": str(first_run.id),
        # Add the specific metrics for the sparql_query_construction run
        "sparql_construction_prompt_tokens": sparql_construction_run.prompt_tokens or 0,
        "sparql_construction_completion_tokens": sparql_construction_run.completion_tokens or 0,
        "sparql_construction_total_tokens": sparql_construction_run.total_tokens or 0,
        "sparql_construction_prompt_cost": sparql_construction_run.prompt_cost or 0,
        "sparql_construction_completion_cost": sparql_construction_run.completion_cost or 0,
        "sparql_construction_total_cost": sparql_construction_run.total_cost or 0,
        # Keep the original metrics too if needed
        "prompt_tokens": first_run.prompt_tokens or 0,
        "completion_tokens": first_run.completion_tokens or 0,
        "total_tokens": first_run.total_tokens or 0,
        "prompt_cost": first_run.prompt_cost or 0,
        "completion_cost": first_run.completion_cost or 0,
        "total_cost": first_run.total_cost or 0,
    }
else:
    print("Warning: Could not find the sparql_query_construction run")
    # Fall back to using the parent run metrics
    result = {
        # Original metrics...
    }
print(result)

# # === BASIC INFORMATION ===
# print("=== BASIC INFORMATION ===")
# print(f"ID: {first_run.id}")
# print(f"Name: {first_run.name}")
# print(f"Run Type: {first_run.run_type}")
# print(f"Status: {first_run.status}")
# print(f"App Path: {first_run.app_path}")

# # === TIMING INFORMATION ===
# print("\n=== TIMING INFORMATION ===")
# print(f"Start Time: {first_run.start_time}")
# print(f"End Time: {first_run.end_time}")
# print(f"First Token Time: {first_run.first_token_time}")
# print(f"Execution Time: {first_run.end_time - first_run.start_time}")

# # === TOKEN AND COST METRICS ===
# print("\n=== TOKEN AND COST METRICS ===")
# print(f"Prompt Tokens: {first_run.prompt_tokens}")
# print(f"Completion Tokens: {first_run.completion_tokens}")
# print(f"Total Tokens: {first_run.total_tokens}")
# print(f"Prompt Cost: ${first_run.prompt_cost}")
# print(f"Completion Cost: ${first_run.completion_cost}")
# print(f"Total Cost: ${first_run.total_cost}")

# # === INPUT/OUTPUT DATA ===
# print("\n=== INPUT/OUTPUT DATA ===")
# print(f"Inputs: {first_run.inputs}")
# print(f"Outputs: {first_run.outputs}")
# print(f"Error: {first_run.error}")

# # === RELATIONSHIP PROPERTIES ===
# print("\n=== RELATIONSHIP PROPERTIES ===")
# print(f"Parent Run ID: {first_run.parent_run_id}")
# print(f"Parent Run IDs: {first_run.parent_run_ids}")
# print(f"Child Run IDs: {first_run.child_run_ids}")
# print(f"Child Runs: {first_run.child_runs}")
# print(f"Trace ID: {first_run.trace_id}")
# print(f"Dotted Order: {first_run.dotted_order}")

# # === METADATA ===
# print("\n=== METADATA ===")
# print(f"Tags: {first_run.tags}")
# print(f"Extra: {first_run.extra}")
# print(f"Serialized: {first_run.serialized}")
# print(f"Events: {first_run.events}")
# print(f"Feedback Stats: {first_run.feedback_stats}")
# print(f"In Dataset: {first_run.in_dataset}")
# print(f"Reference Example ID: {first_run.reference_example_id}")
# print(f"Session ID: {first_run.session_id}")
# print(f"Manifest ID: {first_run.manifest_id}")

AttributeError: 'Run' object has no attribute 'model'

In [1]:
import asyncio
import sys
sys.path.append('/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent')

from experiments.utilities.evaluation import AgentEvaluator


evaluator = AgentEvaluator(
    endpoint_sets={"Uniprot": ["48_glycosylation_sites_and_glycans.ttl"]},
    test=True,
    #experiment_dir="experiments_official"
)

await evaluator.run_all_tests()

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sebastian/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Processing endpoint set: Uniprot

Processed: 48_glycosylation_sites_and_glycans.ttl
Processed 1 files from Uniprot
Saved metadata for 1 files from 1 endpoint sets to:
  - /Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent/experiments/eval/ev_2025-04-12_21-28-15/testset_meta_data.json
Sending question 1/1 to agent...
{'messages': [HumanMessage(content='Retrieve glycosylation sites and glycans on human enzymes (federated with Glyconnect)', additional_kwargs={}, response_metadata={}, id='bbe3a773-6b6e-496c-bfa3-a5873c21f29c')], 'structured_question': StructuredQuestion(intent='access_resources', extracted_classes=['glycosylation sites', 'glycans', 'human enzymes'], extracted_entities=['Glyconnect'], question_steps=['Retrieve glycosylation sites on human enzymes', 'Retrieve glycans on human enzymes', 'Federate with Glyconnect']), 'retrieved_docs': [], 'extracted_entities': [{'matchs': [Document(metadata={'uri': 'http://purl.uniprot.org/taxonomy/625622', 'type': 'l

/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent/.venv/lib/python3.11/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


SP-BLEU 0.05295001712264213
METEOR 0.3269457682381245
F1-score:  0
SP-F1-score:  0
Number of invalid queries(None): 0
Saved query comparison to: /Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent/experiments/eval/ev_2025-04-12_21-28-15/48_comparison.ttl


'/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent/experiments/eval/ev_2025-04-12_21-28-15/evaluation_dataset.json'

# Metric testing

In [3]:
# Create test pairs for eval_pairs function with comments to test clean_sparql_query
test_pairs = [
    (
        """# This is a comment at the beginning of the query
PREFIX rh: <[http://rdf.rhea-db.org/>](http://rdf.rhea-db.org/>)
PREFIX up: <[http://purl.uniprot.org/core/>](http://purl.uniprot.org/core/>) # Comment after a PREFIX declaration

# Query 12 - This is a descriptive comment
# Select all Rhea reactions used to annotate enzyme sequences in UniProtKB
SELECT ?uniprotCount ?rhea ?accession ?equation # Comment after SELECT
WHERE {
  SERVICE <[https://sparql.uniprot.org/sparql>](https://sparql.uniprot.org/sparql>) { # Comment inside the query
    SELECT ?rhea (count(?uniprot) as ?uniprotCount) {
      ?uniprot up:annotation/up:catalyticActivity/up:catalyzedReaction ?rhea . # Technical comment
    }
    GROUP BY ?rhea # Grouping comment
  }
  ?rhea rh:accession ?accession . # Comment with special characters: @#$%^&*()
  ?rhea rh:equation ?equation .
} # Final comment at the end of the query""",
        
        """# [https://sparql.rhea-db.org/sparql](https://sparql.rhea-db.org/sparql)
PREFIX rhea: <[http://rdf.rhea-db.org/>](http://rdf.rhea-db.org/>) # Namespace for Rhea

# This query selects all reactions
# It's a simplified version of the ground truth query
SELECT DISTINCT ?reaction # Get unique reactions only
WHERE {
  ?reaction a rhea:Reaction . # This finds all instances of the Reaction class
  # The following line is commented out but would filter by status
  # ?reaction rhea:status "approved" .
}"""
    ),
    
    # Second example with a different query pair and different comment styles
    (
        """PREFIX CHEBI: <[http://purl.obolibrary.org/obo/CHEBI_>](http://purl.obolibrary.org/obo/CHEBI_>) # Chemical Entities of Biological Interest
PREFIX rh: <[http://rdf.rhea-db.org/>](http://rdf.rhea-db.org/>)
PREFIX skos: <[http://www.w3.org/2004/02/skos/core#>](http://www.w3.org/2004/02/skos/core#>)
PREFIX up: <[http://purl.uniprot.org/core/>](http://purl.uniprot.org/core/>)

# This query finds proteins linked to arachidonate
# CHEBI:32395 is the identifier for arachidonate
SELECT 
    DISTINCT # Ensure unique results
        ?uniprot
        ?uniprotID
        ?recname
        ?gene
        ?chebi
        ?uniprotName # Name from UniProt
WHERE {
  SERVICE <[https://sparql.rhea-db.org/sparql>](https://sparql.rhea-db.org/sparql>) { # Federated query to Rhea
     VALUES (?chebi) { (CHEBI:32395) } # Specify the chemical entity
     ?rhea rh:side/rh:contains/rh:compound ?compound . # Path to compound
     ?compound rh:chebi ?chebi .
     ?chebi up:name ?uniprotName . # Get the name
  }
  ?uniprot up:annotation/up:catalyticActivity/up:catalyzedReaction ?rhea . # Link to reaction
  ?uniprot up:mnemonic ?uniprotID . # Get the ID
  ?uniprot up:recommendedName/up:fullName ?recname .
  OPTIONAL {?uniprot up:encodedBy/skos:prefLabel ?gene .} # Gene name is optional
} # End of query""",
        
        """# This is the predicted query for arachidonate proteins
# It uses different prefixes than the ground truth
PREFIX obo: <[http://purl.obolibrary.org/obo/>](http://purl.obolibrary.org/obo/>) # Ontology prefix
PREFIX rhea: <[http://rdf.rhea-db.org/reaction/>](http://rdf.rhea-db.org/reaction/>) # Different Rhea prefix

SELECT DISTINCT ?protein ?reaction ?ecnumber # Different variables
WHERE {
    # Identify the CHEBI ID for arachidonate
    VALUES ?chebi {obo:CHEBI_32395} # Same entity, different prefix format
    
    # Query Rhea to find reactions associated with arachidonate
    SERVICE <[https://sparql.rhea-db.org/sparql>](https://sparql.rhea-db.org/sparql>) { # Same service endpoint
        ?reaction rhea:side ?side . # Different property path
        ?side rhea:contains ?chebi .
        ?reaction rhea:catalyzedBy ?protein . # Different relationship
    }
    
    # Retrieve EC numbers associated with the reactions
    ?reaction obo:RO_0000057 ?ecnumber . # Additional information
    # ?protein rdfs:label ?name . # This line is commented out
}"""
    )
]

# Example usage in your notebook:
# import sys
# sys.path.append('../utilities')
# from metrics import eval_pairs
# from format import clean_sparql_query
# 
# # Clean the queries before evaluation
# cleaned_pairs = [(clean_sparql_query(gt), clean_sparql_query(pred)) for gt, pred in test_pairs]
# 
# # Run evaluation on cleaned pairs
# results = eval_pairs(cleaned_pairs)
# print(results)

In [2]:
import sys,os


sys.path.append('/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent')
#from metrics import eval_pairs
from experiments.utilities.format import clean_sparql_query

# Clean the queries before evaluation
cleaned_pairs = [(clean_sparql_query(gt), clean_sparql_query(pred)) for gt, pred in test_pairs]

print(test_pairs)
#print(cleaned_pairs)

[('# This is a comment at the beginning of the query\nPREFIX rh: <[http://rdf.rhea-db.org/>](http://rdf.rhea-db.org/>)\nPREFIX up: <[http://purl.uniprot.org/core/>](http://purl.uniprot.org/core/>) # Comment after a PREFIX declaration\n\n# Query 12 - This is a descriptive comment\n# Select all Rhea reactions used to annotate enzyme sequences in UniProtKB\nSELECT ?uniprotCount ?rhea ?accession ?equation # Comment after SELECT\nWHERE {\n  SERVICE <[https://sparql.uniprot.org/sparql>](https://sparql.uniprot.org/sparql>) { # Comment inside the query\n    SELECT ?rhea (count(?uniprot) as ?uniprotCount) {\n      ?uniprot up:annotation/up:catalyticActivity/up:catalyzedReaction ?rhea . # Technical comment\n    }\n    GROUP BY ?rhea # Grouping comment\n  }\n  ?rhea rh:accession ?accession . # Comment with special characters: @#$%^&*()\n  ?rhea rh:equation ?equation .\n} # Final comment at the end of the query', '# [https://sparql.rhea-db.org/sparql](https://sparql.rhea-db.org/sparql)\nPREFIX rhe

In [4]:
import sys,os

sys.path.append('/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent')

from experiments.utilities.metrics import eval_pairs
from experiments.utilities.format import clean_sparql_query

cleaned_pairs = [(clean_sparql_query(gt), clean_sparql_query(pred)) for gt, pred in test_pairs]

eval = eval_pairs(cleaned_pairs)

print(eval)

/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent/.venv/lib/python3.11/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent/.venv/lib/python3.11/site-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


SP-BLEU 9.266129794683117e-79
METEOR 0.19198805586465886
F1-score:  0
SP-F1-score:  0
Number of invalid queries(None): 0
{'SP-BLEU': 9.266129794683117e-79, 'METEOR': 0.19198805586465886, 'f1': 0, 'sp_f1': 0, 'num_none_queries': 0}


In [7]:
identical_query = """# This is a SPARQL query with comments
PREFIX rdf: <[http://www.w3.org/1999/02/22-rdf-syntax-ns#>](http://www.w3.org/1999/02/22-rdf-syntax-ns#>)
PREFIX rdfs: <[http://www.w3.org/2000/01/rdf-schema#>](http://www.w3.org/2000/01/rdf-schema#>) # RDF Schema namespace

# This query finds all classes in the dataset
SELECT DISTINCT ?class ?label # Get classes and their labels
WHERE {
  ?instance a ?class . # This finds all classes used in the dataset
  OPTIONAL { 
    ?class rdfs:label ?label . # Get the label if available
    # Labels make the results more readable
  }
} 
ORDER BY ?class # Sort alphabetically by class URI
LIMIT 100 # Only return the first 100 results
# End of query"""

test_pairs = [
    (identical_query, identical_query),  # Same query for ground truth and prediction
    (identical_query, identical_query)   # Second pair, also identical
]

In [ ]:
#!/usr/bin/env python3
import sys
import os
import json
from datetime import datetime

# Add project root to path
sys.path.append('/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent')

from experiments.utilities.metrics import eval_pairs
from experiments.utilities.format import clean_sparql_query

# Assuming test_pairs is defined in the environment where this script is run
# Clean the SPARQL queries
cleaned_pairs = [(clean_sparql_query(gt), clean_sparql_query(pred)) for gt, pred in test_pairs]

# Run evaluation
evaluation_results = eval_pairs(cleaned_pairs)

# Create output filename with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_path = f"evaluation_results_{timestamp}.json"

# Store both original and cleaned queries
pairs_data = []
for i, ((original_gt, original_pred), (cleaned_gt, cleaned_pred)) in enumerate(zip(test_pairs, cleaned_pairs)):
    pairs_data.append({
        "pair_id": i,
        "original_ground_truth": original_gt,
        "original_prediction": original_pred,
        "cleaned_ground_truth": cleaned_gt,
        "cleaned_prediction": cleaned_pred
    })


results_dict = {
    "metrics": {metric: value for metric, value in evaluation_results.items()},
    "num_pairs_evaluated": len(test_pairs),
    "query_pairs": pairs_data
}

# Write to JSON file
with open(output_path, 'w') as f:
    json.dump(results_dict, f, indent=2)

print(f"Evaluation results saved to: {output_path}")
print(f"Metrics: {evaluation_results}")

SP-BLEU 1.0
METEOR 0.9999794989544467
F1-score:  0
SP-F1-score:  0
Number of invalid queries(None): 0
Evaluation results saved to: evaluation_results_20250412_162049.json
Metrics: {'SP-BLEU': 1.0, 'METEOR': 0.9999794989544467, 'f1': 0, 'sp_f1': 0, 'num_none_queries': 0}


# Result comparision development

In [2]:
import sys
import os

sys.path.append('/Users/sebastian/Documents/Bachelor Thesis/sparql-rag-agent/sparql-rag-agent')

from entity_indexing.endpoint_loader import query_sparql

In [9]:
ground_truth_query = """PREFIX CHEBI: <http://purl.obolibrary.org/obo/CHEBI_>
PREFIX rh: <http://rdf.rhea-db.org/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX up: <http://purl.uniprot.org/core/>


SELECT 
    DISTINCT
        ?uniprot
        ?uniprotID
        ?recname
        ?gene
        ?chebi
        ?uniprotName
WHERE {
  SERVICE <https://sparql.rhea-db.org/sparql> {
     VALUES (?chebi) { (CHEBI:32395) }
     ?rhea rh:side/rh:contains/rh:compound ?compound .
     ?compound rh:chebi ?chebi .
     ?chebi up:name ?uniprotName .
  }
  ?uniprot up:annotation/up:catalyticActivity/up:catalyzedReaction ?rhea .
  ?uniprot up:mnemonic ?uniprotID .
  ?uniprot up:recommendedName/up:fullName ?recname .
  OPTIONAL {?uniprot up:encodedBy/skos:prefLabel ?gene .}
}
LIMIT 100
"""

target_endpoint = "https://sparql.uniprot.org/sparql/"

In [10]:
predicted_query = """PREFIX obo: <http://purl.obolibrary.org/obo/>
PREFIX rhea: <http://rdf.rhea-db.org/reaction/>

SELECT DISTINCT ?protein ?reaction ?ecnumber
WHERE {
    # Identify the CHEBI ID for arachidonate
    VALUES ?chebi {obo:CHEBI_32395}
    
    # Query Rhea to find reactions associated with arachidonate
    SERVICE <https://sparql.rhea-db.org/sparql> {
        ?reaction rhea:side ?side .
        ?side rhea:contains ?chebi .
        ?reaction rhea:catalyzedBy ?protein .
    }
    
    # Retrieve EC numbers associated with the reactions
    ?reaction obo:RO_0000057 ?ecnumber .
}"""

In [11]:
results_ground_truth = query_sparql(ground_truth_query, target_endpoint)
print(results_ground_truth)

{'head': {'vars': ['uniprot', 'uniprotID', 'recname', 'gene', 'chebi', 'uniprotName']}, 'results': {'bindings': [{'uniprotName': {'type': 'literal', 'value': '(5Z,8Z,11Z,14Z)-eicosatetraenoate'}, 'chebi': {'type': 'uri', 'value': 'http://purl.obolibrary.org/obo/CHEBI_32395'}, 'gene': {'type': 'literal', 'value': 'CYP1A2'}, 'uniprot': {'type': 'uri', 'value': 'http://purl.uniprot.org/uniprot/O77809'}, 'uniprotID': {'type': 'literal', 'value': 'CP1A2_MACFA'}, 'recname': {'type': 'literal', 'value': 'Cytochrome P450 1A2'}}, {'uniprotName': {'type': 'literal', 'value': '(5Z,8Z,11Z,14Z)-eicosatetraenoate'}, 'chebi': {'type': 'uri', 'value': 'http://purl.obolibrary.org/obo/CHEBI_32395'}, 'gene': {'type': 'literal', 'value': 'CYP1A2'}, 'uniprot': {'type': 'uri', 'value': 'http://purl.uniprot.org/uniprot/O77810'}, 'uniprotID': {'type': 'literal', 'value': 'CP1A2_CALJA'}, 'recname': {'type': 'literal', 'value': 'Cytochrome P450 1A2'}}, {'uniprotName': {'type': 'literal', 'value': '(5Z,8Z,11Z,14

In [12]:
results_predicted = query_sparql(predicted_query, target_endpoint)
print(results_predicted)

error


In [13]:
import pandas as pd

df = pd.DataFrame(ground_truth_query)

display(df.head())

ValueError: DataFrame constructor not properly called!